<script>
    document.querySelector('head').innerHTML += '<style>.slides { zoom: 1.0 !important; }</style>';
    document.querySelector('blockquote').innerHTML += '<style>.slides { width: 100% !important; }</style>';
</script>
# Тренды & дайджесты
---
Результаты и код

## Система сбора статей
__Задача:__ реализовать систему сбора новостных статей

In [1]:
# TODO: код Игоря

In [2]:
# а пока так:
import numpy as np
import pandas as pd

df = pd.read_csv('sample_dataset.tsv', sep='\t')
df.head(3)

,Date,Link,Text,Title
0,2022-10-08 04:53:00,https://lenta.ru/news/2022/10/08/saud/,Глава МИД Саудовской Аравии Адель аль-Джубейр ...,В Саудовской Аравии ответили на претензии США ...
1,2022-10-08 00:37:51,https://lenta.ru/news/2022/10/08/mvf/,Исполнительный совет Международного валютного ...,"МВФ выделил 1,3 миллиарда долларов Украине"
2,2022-10-07 21:31:00,https://lenta.ru/news/2022/10/07/sakhalin/,Президент России Владимир Путин подписал указ ...,Путин подписал указ о создании нового оператор...


## Выделение новостных трендов
__Задача:__ выделить тренды с помощью нейросетевых методов

### Группируем статьи по их сходству:
-  Лемматизируем наши статьи, выделяем $n$-граммы.
-  Используем предобученные эмбеддинги для русского языка, которые предлагает __spaCy__.
-  Сходство считаем через *cosine similarity* (он же *Коэффициент Отиаи*).

In [28]:
POS_TAGS = ['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']
VOWELS = frozenset('аоиыуэ')

In [29]:
import gensim

def count_vowels(text):
    return sum(letter in VOWELS for letter in text)

def lemmatize(articles):

    # remove e-mails and url links
    articles = articles.replace(r'\S*@\S*\s?', '', regex=True)
    articles = articles.replace(r'http\S+', '', regex=True)
    articles = articles.replace(r'www.\S+', '', regex=True)

    # fix punctuation
    articles = articles.replace(r'\.([а-яА-Я])', r'. \g<1>', regex=True)

    # remove all non-alphabetic characters
    articles = articles.replace(r'[^а-яА-Я.]+', ' ', regex=True)

    # lowercase
    articles = articles.str.lower()

    docs = list(NLP.pipe(articles, batch_size=len(articles)))

    tokens = [[token.lemma_ for token in article if token.pos_ in POS_TAGS] for article in docs]
    sentence_count = [len(list(article.sents)) for article in docs]
    syllable_count = [count_vowels(article) for article in articles]  # in russian, no. of vowels ~= no. of syllables

    return tokens, sentence_count, syllable_count

def form_ngrams(articles: pd.Series) -> pd.Series:
    '''
    Substitutes collocations with respective n-grams where it can be applied.
    Used in eval_data_4_role()

    :param articles: Series containing the necessary tokenized articles
    :return: Series with n-grams added
    '''
    bigram = gensim.models.Phrases(articles)
    trigram = gensim.models.Phrases(bigram[articles])

    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    temp = pd.Series(0, index=np.arange(len(articles)))
    for i, entry in articles.items():
        temp[i] = bigram_mod[entry]
        temp[i] = trigram_mod[bigram_mod[entry]]

    return temp

In [30]:
df['Text_Clean'], _, _ = lemmatize(df.Text)
df['Text_Clean'] = form_ngrams(df['Text_Clean'])
df['Text_Clean'] = df.Text_Clean.str.join(' ')

In [109]:
THRESHOLD = 0.915

Группируем статьи по спискам индексов

In [110]:
import spacy
from itertools import chain

NLP = spacy.load('ru_core_news_lg')

docs = list(NLP.pipe(df.Text_Clean, batch_size=len(df.Text_Clean)))

def get_similarity(index1, index2) -> float:
    return docs[index1].similarity(docs[index2])

res = []
added_articles = set()

for i in range(len(df)):
    for j in range(i + 1, len(df)):
        if get_similarity(i, j) > THRESHOLD:
            added_articles.add(i)
            added_articles.add(j)

            if i and j in chain(*res): continue

            new_group = {i, j}
            for group in res:
                if i in group:
                    flag = True
                    for element in group:
                        if get_similarity(element, j) < THRESHOLD:
                            new_group.remove(i)
                            flag = False
                    if flag:
                        group.append(j)
                        new_group.clear()
                        break
                elif j in group:
                    flag = True
                    for element in group:
                        if get_similarity(element, i) < THRESHOLD:
                            new_group.remove(j)
                            flag = False
                    if flag:
                        group.append(i)
                        new_group.clear()
                        break

            if new_group:
                res.append(list(new_group))
            # print(res)

for i in list(df.index.values):
    if i not in added_articles:
        res.append([i])

In [111]:
res.sort(key=len, reverse=True)

Вывод матрицы сходств (для красоты)

In [112]:
SLICE_SIZE = 13

In [113]:
indices = list(df.index.values)[:SLICE_SIZE]
rowConst, colConst = [tmp.T for tmp in np.meshgrid(indices, indices)]
matrix = pd.DataFrame(np.vectorize(get_similarity)(rowConst, colConst), index=indices, columns=indices)
matrix.update(matrix.applymap('{:,.3f}'.format))

In [114]:
matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.000,0.773,0.786,0.770,0.746,0.757,0.774,0.796,0.834,0.647,0.823,0.683,0.801
1,0.773,1.000,0.812,0.734,0.815,0.812,0.811,0.807,0.798,0.744,0.818,0.808,0.797
2,0.786,0.812,1.000,0.807,0.766,0.795,0.839,0.855,0.828,0.827,0.849,0.841,0.851
3,0.770,0.734,0.807,1.000,0.703,0.802,0.850,0.868,0.819,0.733,0.823,0.722,0.833
4,0.746,0.815,0.766,0.703,1.000,0.750,0.825,0.759,0.824,0.692,0.856,0.754,0.802
5,0.757,0.812,0.795,0.802,0.750,1.000,0.856,0.881,0.843,0.754,0.779,0.728,0.895
6,0.774,0.811,0.839,0.850,0.825,0.856,1.000,0.879,0.880,0.808,0.864,0.817,0.883
7,0.796,0.807,0.855,0.868,0.759,0.881,0.879,1.000,0.829,0.759,0.814,0.765,0.897
8,0.834,0.798,0.828,0.819,0.824,0.843,0.880,0.829,1.000,0.735,0.896,0.725,0.903
9,0.647,0.744,0.827,0.733,0.692,0.754,0.808,0.759,0.735,1.000,0.765,0.897,0.765


Получили группы статей по темам. Чем больше статей на одну тему &mdash; тем она популярнее сейчас.

__Проблема:__ как среди одинаковых статей выбрать лучшую?

In [118]:
df.iloc[res[2]]

,Date,Link,Text,Title,Lexical_Diversity,FRE,Neutral,Positive,Negative,Score,Text_Clean
18,2022-10-06 14:34:00,https://lenta.ru/news/2022/10/06/neffft/,Решение Евросоюза (ЕС) о введении потолка цен ...,Кремль оценил решение ЕС о потолке цен на росс...,0.778761,0.333017,0.180738,0.067849,0.751413,0.608952,решение евросоюз ес введение_потолок цена нефт...
21,2022-10-06 12:20:00,https://lenta.ru/news/2022/10/06/potolok/,Европейская комиссия утвердила новый пакет сан...,Европа согласовала потолок цен для российской ...,0.761364,0.239691,0.823185,0.144318,0.032497,1.936061,европейский комиссия утвердить новый пакет сан...


### Введем в рассмотрение следующие метрики:
- лексическое разнообразие<sup>1</sup>:
    <p style="text-align: center;">$\frac{\text{unique lexemes}}{\text{total lexemes}}$,</p>
- индекс удобочитаемости Флеша (FRE)<sup>2</sup>:
    <p style="text-align: center;">$FRE = 206.835 − 1.52\frac{\text{total words}}{\text{total sentences}} − 65.14 \frac{\text{total syllables}}{\text{total words}}$,</p>
- тональность текста:
    - пессимизируем негативные статьи;
    - оптимизируем нейтральные (реже &mdash; позитивные) статьи.

Считаем метрики 1-2

In [25]:
def evaluate(articles):
    tokens, total_sentences, total_syllables = lemmatize(articles)

    lexical_diversity = np.fromiter((len(set(tokens)) / len(tokens) for tokens in tokens), float)

    # Flesch Reading Ease
    fre = np.fromiter((206.835 - 1.52 * (len(w) / sen) - 65.14 * (syl / len(w)) for w, sen, syl in zip(tokens, total_sentences, total_syllables)), float)
    fre = (fre - np.min(fre)) / (121.22 - np.min(fre))  # 121.22 is the highest possible score;
                                                        # there's no lower bound, so we use the lowest score we got as a reference
    return tokens, lexical_diversity, fre

In [26]:
df["Text_Clean"], df["Lexical_Diversity"], df["FRE"] = evaluate(df.Text)

In [27]:
df

,Date,Link,Text,Title,Lexical_Diversity,FRE,Neutral,Positive,Negative,Score,Text_Clean
0,2022-10-08 04:53:00,https://lenta.ru/news/2022/10/08/saud/,Глава МИД Саудовской Аравии Адель аль-Джубейр ...,В Саудовской Аравии ответили на претензии США ...,0.795181,0.342219,0.180781,0.067882,0.751337,0.634726,"[глава, мид, саудовский, аравия, адель, джубей..."
1,2022-10-08 00:37:51,https://lenta.ru/news/2022/10/08/mvf/,Исполнительный совет Международного валютного ...,"МВФ выделил 1,3 миллиарда долларов Украине",0.776596,0.151390,0.824889,0.142081,0.033030,1.861926,"[исполнительный, совет, международный, валютны..."
2,2022-10-07 21:31:00,https://lenta.ru/news/2022/10/07/sakhalin/,Президент России Владимир Путин подписал указ ...,Путин подписал указ о создании нового оператор...,0.700000,0.213808,0.180691,0.067857,0.751453,0.410903,"[президент, россия, владимир, путин, подписать..."
3,2022-10-07 19:11:00,https://lenta.ru/news/2022/10/07/semi_ch/,Администрация президента США объявила о новых ...,США нашли способ ударить по доступу Китая к кл...,0.820513,0.045299,0.180675,0.067848,0.751478,0.362857,"[администрация, президент, сша, объявить, новы..."
4,2022-10-07 17:00:00,https://lenta.ru/news/2022/10/07/ecbbanki/,Европейский центробанк (ЕЦБ) ужесточает требов...,От европейских банков потребовали экономии,0.805310,0.244684,0.817153,0.150579,0.032268,1.985458,"[европейский, центробанк, ецб, ужесточать, тре..."
5,2022-10-07 15:55:00,https://lenta.ru/news/2022/10/07/natzval/,"Страны, которые входят в СНГ, переходят на рас...",Путин объяснил переход стран СНГ на расчеты в ...,0.849315,0.165046,0.833623,0.129364,0.037013,1.940336,"[страна, входить, снг, переходить, расчёт, нац..."
6,2022-10-07 15:22:40,https://lenta.ru/news/2022/10/07/genius/,Президент Белоруссии Александр Лукашенко подпи...,Лукашенко запретил отменять скидки по акции,0.860927,0.237882,0.180701,0.067861,0.751438,0.595932,"[президент, белоруссия, александр, лукашенко, ..."
7,2022-10-07 13:48:00,https://lenta.ru/news/2022/10/07/dadut/,Россия и Белоруссия достигли окончательной дог...,В Белоруссии раскрыли сумму ожидаемых от Росси...,0.857143,0.053754,0.824107,0.143030,0.032863,1.845170,"[россия, белоруссия, достигнуть, окончательный..."
8,2022-10-07 13:28:50,https://lenta.ru/news/2022/10/07/borrell/,В ЕС нашли объяснение решению ОПЕК+ снизить до...,В ЕС нашли объяснение решению ОПЕК+ снизить до...,0.814570,0.322413,0.180665,0.067837,0.751498,0.633987,"[ес, найти, объяснение, решение, опек, снизить..."
9,2022-10-07 12:37:00,https://lenta.ru/news/2022/10/07/garantii_putin/,Российский президент Владимир Путин подписал з...,Путин подписал закон о трудовых правах мобилиз...,0.767742,0.000000,0.180686,0.067854,0.751461,0.264821,"[российский, президент, владимир, путин, подпи..."


Считаем тональность / сентимент

In [14]:
import torch
from transformers import BertTokenizerFast, AutoModelForSequenceClassification

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment')

def predict_sentiment(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)

    return pd.Series(predicted.detach().numpy().flatten())

In [15]:
df[['Neutral', 'Positive', 'Negative']] = df.apply(lambda row: predict_sentiment(row['Text']), axis=1)

<p style="text-align: center;">$\text{Score} = \text{Lexical_Diversity} + \text{FRE} + \text{Neutral} + \text{Positive} - \text{Negative}$,</p>

In [16]:
df['Score'] = df.Lexical_Diversity + df.FRE + df.Neutral + df.Positive - df.Negative

### Результат
Топ-3 новости из датасета

In [17]:
TOP = 3

In [18]:
indices_sorted = list(df.sort_values(by=['Score'], ascending=False).index)

for i in range(TOP):
    best_in_group_idx = next(match for match in indices_sorted if match in res[i])
    print(df[['Title', 'Score']].loc[best_in_group_idx])

Title    В России отреагировали на введение восьмого па...
Score                                             1.750574
Name: 16, dtype: object
Title    В ЕС нашли объяснение решению ОПЕК+ снизить до...
Score                                             0.633987
Name: 8, dtype: object
Title    Европа согласовала потолок цен для российской ...
Score                                             1.936061
Name: 21, dtype: object


In [19]:
# df[['Title', 'Score']].sort_values(by=['Score'], ascending=False).head(3)

## Формирование новостного дайджеста
__Задача:__ вывести набор кратких изложений статей, которые были выделены в задаче о трендах.

Создание краткого изложения текста — это задача text summarization.

Существует два подхода к суммаризации текста: экстракция и абстракция.

__Экстракция__ — это выделение наиболее значимых частей текста. При этом не происходит изменения исходного текста, алгоритм лишь выделяет части этого текста и склеивает их. Такой подход применяется в библиотеках Sumy (в ней имплементированы алгоритмы LexRank, Luhn, LSA, TextRank) и Gensim (TextRank).

__Абстракция__ — это обобщение, пересказ исходного текста. При этом текст-вывод может не содержаться дословно в тексте-вводе: он генерируется с нуля. Для реализации этого подхода используются рекуррентные и свёрточные нейронные сети, обеспечивающие natural language understanding и natural language generation.

Выбор подхода зависит от условий решаемой задачи: экстрактивные алгоритмы проще в реализации, а абстрактивные теоретически должны давать более качественный результат.

Для сравнения результатов суммаризации с применением разных подходов возьмём Sumy для экстракции, MBart и ruGPT-3 для абстракции.

### MBart
Sequence-to-sequence модель из двух рекуррентных нейросетей, предобученная на датасете из статей с Газеты.ру. 

In [20]:
def summarize(article):
    from transformers import MBartTokenizer, MBartForConditionalGeneration

    model_name = "IlyaGusev/mbart_ru_sum_gazeta"
    tokenizer = MBartTokenizer.from_pretrained(model_name)
    model = MBartForConditionalGeneration.from_pretrained(model_name)

    article_text = article

    input_ids = tokenizer(
        [article_text],
        max_length=25,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        repetition_penalty=3.0
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    print(summary)

Для тестирования на вход была дана статья: "Глава МИД Саудовской Аравии Адель аль-Джубейр в интервью <mark>Fox News</mark> ответил на <mark>претензии США из-за сокращения добычи нефти</mark>. Он подчеркнул, что высокие цены на бензин в США — не вина Саудовской Аравии, а следствие нехватки нефтеперерабатывающих заводов (НПЗ) в самих Штатах. «У вас дефицит переработки, вы не строили нефтеперерабатывающие заводы десятилетиями», — заявил министр. При этом аль-Джубейр заверил, что Саудовская Аравия не ставила целью навредить США, а идея об этом «целиком и полностью неверна». Ранее президент США Джо Байден обвинил Россию и Саудовскую Аравию в росте цен на бензин в стране после того, как ОПЕК+ решила резко сократить добычу нефти с ноября текущего года, об этом стало известно 5 октября. Страны должны будут уменьшить производство на два миллиона баррелей в сутки."

Результат с ограничением 20 слов: "<mark>Westmail News</mark> опубликовало интервью, в котором глава МИД Саудовской Аравии Адель аль-Джубейр рассказал об <mark>отношениях между Москвой и Вашингтоном</mark>."

Результат с ограничением 25 слов: "<mark>States Weekly</mark> опубликовало интервью, в котором глава МИД Саудовской Аравии Адель аль-Джубейр ответил на <mark>обвинения в коррупции</mark>."

__Вывод:__ работает, но бесполезно из-за ошибок в понимании и генерации текста.

### ruGPT-3
Generative Pre-Trained Transformer, обученный на большом русскоязычном датасете.

Я потратила несколько часов на то, чтобы разобраться в коде и настроить окружение, но так и не настроила.

__Вывод:__ возможно, работает, но в ряде случаев бесполезно из-за слишком долгой и сложной первичной локальной настройки. Можно использовать в долгоиграющих системах, но вряд ли — для хакатонов (или с предварительной подготовкой). 

Есть статья о применении ruGPT-3 к задаче о суммаризации русскоязычных текстов<sup>3</sup>. В ней приведены примеры работы модели, которые демонстрируют, что качество её краткого пересказа приемлемое, но не на уровне пересказа человеком.

### Sumy
Библиотека, включающая в себя алгоритмы суммаризации текстов.

Я попробовала алгоритмы LexRank, TextRank, Luhn и LSA. По моему впечатлению, более точные и содержательные изложения делает Luhn, его и возьмём в конечный код. Внутри библиотеки алгоритмы вызываются почти одинаково, разница лишь в названии вызываемых функций.

In [22]:
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.luhn import LuhnSummarizer

def digest(article: str, n=3):
    my_parser = PlaintextParser.from_string(article, Tokenizer('russian'))
    
    luhn_summarizer = LuhnSummarizer()
    luhn_summary = luhn_summarizer(my_parser.document, n)
    digest_sentences = "Luhn summary: "
    for sentence in luhn_summary:
        digest_sentences += str(sentence) + " "

    return digest_sentences

__Входной текст для тестирования__:
><span style="font-size: 0.75em;">Бывшая студентка из штата Пенсильвания, США, решила скрыть, что ее отчислили из вуза, и инсценировала собственное похищение. Об этом сообщает NBC News. 23-летняя Клои Стайн отправила своему возлюбленному сообщение, что по дороге с работы ее остановил полицейский. После этого девушка перестала выходить на связь. Затем родственники нашли ее брошенный автомобиль и обратились в полицию. Правоохранительные органы организовали масштабные поиски Стайн, был задействован вертолет. К вечеру следующего дня ее обнаружили в городе Дженнет. Девушка объяснила, что ее похитил неизвестный, который представился полицейским. Мужчина якобы, угрожая оружием, завязал ей глаза и увез в неизвестном направлении. Сотрудники полиции обратили внимание на нестыковки в рассказе Стайн. При дальнейших расспросах девушка призналась, что все произошедшее — выдумка. Инсценировка похищения потребовалась, чтобы скрыть, что она больше не учится в Университете штата Пенсильвания. Все это время она скрывала, что ее отчислили еще осенью 2018 года. Стайн задержали по обвинению в совершении четырех правонарушений, в том числе ложном сообщении о преступлении и вмешательстве в деятельность правоохранительных органов. Ранее сообщалось, что гражданин Южной Кореи, находившийся в Таиланде нелегально, инсценировал свое похищение ради денег на расходы для девушки. Он просрочил тайскую визу на 497 дней и сообщил родителям, что его похитили.</span>

  __Luhn summary__:
  >Бывшая студентка из штата Пенсильвания, США, решила скрыть, что ее отчислили из вуза, и инсценировала собственное похищение. Инсценировка похищения потребовалась, чтобы скрыть, что она больше не учится в Университете штата Пенсильвания. Все это время она скрывала, что ее отчислили еще осенью 2018 года.

Как работает Luhn:

1) Делаем стемматизацию текста, удаляем стоп-слова.

2) Сортируем слова по частоте встречаемости.

3) Раздаём веса предложениям в тексте по следующему принципу: если в предложении встречается много наиболее частотных слов с расстоянием в 4-5 слов между ними, то предложению даётся более высокий вес.

4) Выводим n предложений с наибольшим весом.

Алгоритм был представлен сотрудником IBM Гансом Питером Луном в 1958 году и сейчас считается несколько устаревшим (а мне понравился!).

## Источники

[1]: Bird, S., Klein, E., & Loper, E. (2009). Natural language processing with Python: analyzing text with the natural language toolkit. O'Reilly Media.

[2]: Оборнева, И. В. (2005). Математическая модель оценки учебных текстов. Вестник МГПУ. Серия «Информатика и информатизация образования». –М.: МГПУ, (1), 4.

[3]: Alexandr, N., Irina, O., Tatyana, K., Inessa, K., & Arina, P. (2021). Fine-tuning GPT-3 for Russian Text Summarization. In Data Science and Intelligent Systems: Proceedings of 5th Computational Methods in Systems and Software 2021, Vol. 2 (pp. 748-757). Springer International Publishing.